In [1]:
import pandas as pd
import numpy as np
import os
import fileinput
import sys

import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell

# pretty print all cell's output and not just the last one
InteractiveShell.ast_node_interactivity = "all"

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

In [2]:
def transfer_to_all_sources(origin, destination, data_dir='/ptmp/mpa/mglatzle/TNG_f_esc'):
    run_dir_in = os.path.join(data_dir, origin, 'run', 'L35n2160TNG')
    run_dir_out = os.path.join(data_dir, destination, 'run', 'L35n2160TNG')
    run_dir_out_input = os.path.join(data_dir, destination, 'input', 'L35n2160TNG')
    snap_dirs = ['sn013', 'sn008', 'sn004']
    for snap in snap_dirs:
        snap_dir_in = os.path.join(run_dir_in, snap)
        snap_dir_out = os.path.join(run_dir_out, snap)
        
        snap_dir_out_input = os.path.join(run_dir_out_input, snap)
        for halo in os.listdir(snap_dir_in):
            halo_dir_in = os.path.join(snap_dir_in, halo)
            halo_dir_out = os.path.join(snap_dir_out, halo)
            
            output_dir_in = os.path.join(halo_dir_in, 'Output')
            
            halo_dir_out_input = os.path.join(snap_dir_out_input, halo)

            config_dir_out = os.path.join(halo_dir_out, 'config')
            last_rt_in = os.path.join(output_dir_in, 'phys_ic00_rt05.out')
            if os.path.isfile(last_rt_in):
                print(f'working on halo {halo} in snap {snap}')
                if not os.path.isdir(halo_dir_out):
                    os.system(f'cp -r {halo_dir_in} {snap_dir_out}')
                else:
                    print(f'{halo} already exists')

In [3]:
def correct_fesc_file(origin, destination, data_dir='/ptmp/mpa/mglatzle/TNG_f_esc', from_freya=True):
    fesc_file = os.path.join(data_dir, destination, '00_f_esc.txt')
    for line in fileinput.input(fesc_file, inplace=1):
        line = line.replace(origin, destination)
        sys.stdout.write(line)
    fileinput.close()
    return

In [16]:
def correct_paths(origin, destination, with_dust=False, data_dir='/ptmp/mpa/mglatzle/TNG_f_esc'):
    run_dir_out = os.path.join(data_dir, destination, 'run', 'L35n2160TNG')
    run_dir_out_input = os.path.join(data_dir, destination, 'input', 'L35n2160TNG')
    
    init_path = os.path.join(data_dir, destination, 'bin', 'CRASH_5.4.3v09.in')
    modules_path = os.path.join(data_dir, destination, 'config', 'Modules')
    random_path = os.path.join(data_dir, destination, 'config', 'SYS_RANDOM.in')
    snap_dirs = ['sn013', 'sn008', 'sn004']
    for snap in snap_dirs:
        snap_dir_out = os.path.join(run_dir_out, snap)
        
        snap_dir_out_input = os.path.join(run_dir_out_input, snap)
        for halo in os.listdir(snap_dir_out):
            print(f'Working on {halo} in snap {snap}')
            halo_dir_out = os.path.join(snap_dir_out, halo)
            
            
            halo_dir_out_input = os.path.join(snap_dir_out_input, halo)

            config_dir_out = os.path.join(halo_dir_out, 'config')
            
            simul_file = os.path.join(config_dir_out, 'SIMULATION.in')
            
            for line in fileinput.input(simul_file, inplace=1):
                if (line[:8] == "'IN_DIR=") or (line[:9] == "'OUT_DIR="):
                    line = line.replace(origin, destination)
                sys.stdout.write(line)
            fileinput.close()
            
            input_dir = os.path.join(halo_dir_out, 'Input')
            old_init = os.path.join(halo_dir_out, 'SYS_INIT.in')
            old_modules = os.path.join(halo_dir_out, 'config', 'Modules')
            old_random = os.path.join(halo_dir_out, 'config', 'SYS_RANDOM.in')
            
            source_file = os.path.join(input_dir, 'sources_ic00.in')
            simul_file = os.path.join(input_dir, 'simul_ic00.in')
            dens_file = os.path.join(input_dir, 'dens_ic00.in')
            ion_file = os.path.join(input_dir, 'ion.in')
            temp_file = os.path.join(input_dir, 'temp_ic00.in')
            if with_dust:
                dust_dir = os.path.join(input_dir, 'dust')
            
            source_file_sources = os.path.join(halo_dir_out_input, 'sources_ic00.in')
            dens_file_sources = os.path.join(halo_dir_out_input, 'dens_ic00.in')
            simul_file_sources = os.path.join(halo_dir_out_input, 'simul_ic00.in')
            ion_file_sources = os.path.join(halo_dir_out_input, 'ion.in')
            temp_file_sources = os.path.join(halo_dir_out_input, 'temp_ic00.in')
            if with_dust:
                dust_dir_sources = os.path.join(halo_dir_out_input, 'dust')
            
            os.system(f'rm {source_file}')
            os.system(f'rm {simul_file}')
            os.system(f'rm {dens_file}')
            os.system(f'rm {ion_file}')
            os.system(f'rm {temp_file}')
            os.system(f'rm {old_init}')
            os.system(f'rm {old_modules}')
            os.system(f'rm {old_random}')
            if with_dust:
                os.system(f'rm {dust_dir}')
            
            os.system(f'ln -s {source_file_sources} {input_dir}')
            os.system(f'ln -s {dens_file_sources} {input_dir}')
            os.system(f'ln -s {simul_file_sources} {input_dir}')
            os.system(f'ln -s {ion_file_sources} {input_dir}')
            os.system(f'ln -s {temp_file_sources} {input_dir}')
            if with_dust:
                os.system(f'ln -s {dust_dir_sources} {input_dir}')
            os.system(f'ln -s {init_path} {old_init}')
            os.system(f'ln -s {modules_path} {old_modules}')
            os.system(f'ln -s {random_path} {old_random}')

In [5]:
origin = 'new_main'
destination = 'esc_analysis'
transfer_to_all_sources(origin=origin, destination=destination)

working on halo g18024 in snap sn013
g18024 already exists
working on halo g5025 in snap sn013
g5025 already exists
working on halo g36099 in snap sn013
g36099 already exists
working on halo g3205 in snap sn013
working on halo g5444 in snap sn013
g5444 already exists
working on halo g7085 in snap sn013
g7085 already exists
working on halo g5265 in snap sn013
g5265 already exists
working on halo g6715 in snap sn013
g6715 already exists
working on halo g6955 in snap sn013
g6955 already exists
working on halo g14147 in snap sn013
g14147 already exists
working on halo g2324 in snap sn013
g2324 already exists
working on halo g10928 in snap sn013
g10928 already exists
working on halo g34620 in snap sn013
g34620 already exists
working on halo g41356 in snap sn013
g41356 already exists
working on halo g2456 in snap sn013
g2456 already exists
working on halo g13062 in snap sn013
g13062 already exists
working on halo g24614 in snap sn013
g24614 already exists
working on halo g22417 in snap sn013

working on halo g11352 in snap sn013
g11352 already exists
working on halo g103325 in snap sn013
g103325 already exists
working on halo g7950 in snap sn013
g7950 already exists
working on halo g7348 in snap sn013
g7348 already exists
working on halo g11382 in snap sn013
g11382 already exists
working on halo g41732 in snap sn013
g41732 already exists
working on halo g14014 in snap sn013
g14014 already exists
working on halo g4334 in snap sn013
working on halo g6399 in snap sn013
g6399 already exists
working on halo g30405 in snap sn013
g30405 already exists
working on halo g5080 in snap sn013
g5080 already exists
working on halo g28069 in snap sn013
g28069 already exists
working on halo g9153 in snap sn013
g9153 already exists
working on halo g2276 in snap sn013
working on halo g9521 in snap sn013
g9521 already exists
working on halo g4239 in snap sn013
g4239 already exists
working on halo g8874 in snap sn013
g8874 already exists
working on halo g16987 in snap sn013
g16987 already exist

working on halo g11721 in snap sn013
g11721 already exists
working on halo g25768 in snap sn013
g25768 already exists
working on halo g11294 in snap sn013
g11294 already exists
working on halo g8641 in snap sn013
g8641 already exists
working on halo g14428 in snap sn013
g14428 already exists
working on halo g22315 in snap sn013
g22315 already exists
working on halo g3222 in snap sn013
g3222 already exists
working on halo g43427 in snap sn013
g43427 already exists
working on halo g15682 in snap sn013
g15682 already exists
working on halo g46676 in snap sn013
g46676 already exists
working on halo g3191 in snap sn013
g3191 already exists
working on halo g14510 in snap sn013
g14510 already exists
working on halo g15268 in snap sn013
g15268 already exists
working on halo g2663 in snap sn013
g2663 already exists
working on halo g17816 in snap sn013
g17816 already exists
working on halo g7720 in snap sn013
working on halo g6579 in snap sn013
g6579 already exists
working on halo g55970 in snap

working on halo g4435 in snap sn013
g4435 already exists
working on halo g4675 in snap sn013
g4675 already exists
working on halo g14414 in snap sn013
g14414 already exists
working on halo g27096 in snap sn013
g27096 already exists
working on halo g18964 in snap sn013
g18964 already exists
working on halo g58598 in snap sn013
g58598 already exists
working on halo g1796 in snap sn013
g1796 already exists
working on halo g27777 in snap sn013
g27777 already exists
working on halo g1758 in snap sn013
working on halo g4406 in snap sn013
g4406 already exists
working on halo g5819 in snap sn013
g5819 already exists
working on halo g5650 in snap sn013
g5650 already exists
working on halo g4535 in snap sn013
g4535 already exists
working on halo g20318 in snap sn013
g20318 already exists
working on halo g2715 in snap sn013
working on halo g7918 in snap sn013
g7918 already exists
working on halo g2457 in snap sn013
working on halo g13869 in snap sn013
g13869 already exists
working on halo g1637 i

working on halo g12747 in snap sn013
g12747 already exists
working on halo g13160 in snap sn013
g13160 already exists
working on halo g21420 in snap sn013
g21420 already exists
working on halo g752 in snap sn013
working on halo g6726 in snap sn013
g6726 already exists
working on halo g22749 in snap sn013
g22749 already exists
working on halo g22096 in snap sn013
g22096 already exists
working on halo g21230 in snap sn013
g21230 already exists
working on halo g25694 in snap sn013
g25694 already exists
working on halo g14177 in snap sn013
g14177 already exists
working on halo g5917 in snap sn013
g5917 already exists
working on halo g2412 in snap sn013
working on halo g10933 in snap sn013
g10933 already exists
working on halo g4589 in snap sn013
working on halo g9819 in snap sn013
g9819 already exists
working on halo g3798 in snap sn013
g3798 already exists
working on halo g1327 in snap sn013
working on halo g5307 in snap sn013
g5307 already exists
working on halo g3201 in snap sn013
g3201

...

working on halo g8652 in snap sn004
g8652 already exists
working on halo g1390 in snap sn004
g1390 already exists
working on halo g1061 in snap sn004
g1061 already exists
working on halo g2923 in snap sn004
g2923 already exists
working on halo g8226 in snap sn004
g8226 already exists
working on halo g1248 in snap sn004
g1248 already exists
working on halo g616 in snap sn004
g616 already exists
working on halo g1837 in snap sn004
g1837 already exists
working on halo g3230 in snap sn004
g3230 already exists
working on halo g798 in snap sn004
g798 already exists
working on halo g2480 in snap sn004
g2480 already exists
working on halo g1129 in snap sn004
g1129 already exists
working on halo g2282 in snap sn004
g2282 already exists
working on halo g663 in snap sn004
g663 already exists
working on halo g3701 in snap sn004
g3701 already exists
working on halo g1053 in snap sn004
g1053 already exists
working on halo g1176 in snap sn004
g1176 already exists
working on halo g2996 in snap sn004
g

working on halo g858 in snap sn004
g858 already exists
working on halo g7186 in snap sn004
g7186 already exists
working on halo g723 in snap sn004
g723 already exists
working on halo g3994 in snap sn004
g3994 already exists
working on halo g1929 in snap sn004
g1929 already exists
working on halo g8807 in snap sn004
g8807 already exists
working on halo g1046 in snap sn004
g1046 already exists
working on halo g259 in snap sn004
g259 already exists
working on halo g18276 in snap sn004
g18276 already exists
working on halo g2866 in snap sn004
g2866 already exists
working on halo g2012 in snap sn004
g2012 already exists
working on halo g124 in snap sn004
g124 already exists
working on halo g5147 in snap sn004
g5147 already exists
working on halo g5749 in snap sn004
g5749 already exists
working on halo g11923 in snap sn004
g11923 already exists
working on halo g4394 in snap sn004
g4394 already exists
working on halo g171 in snap sn004
g171 already exists
working on halo g1468 in snap sn004
g

working on halo g8247 in snap sn004
g8247 already exists
working on halo g2804 in snap sn004
g2804 already exists
working on halo g2054 in snap sn004
g2054 already exists
working on halo g326 in snap sn004
g326 already exists
working on halo g972 in snap sn004
g972 already exists
working on halo g2627 in snap sn004
g2627 already exists
working on halo g786 in snap sn004
g786 already exists
working on halo g651 in snap sn004
g651 already exists
working on halo g5429 in snap sn004
g5429 already exists
working on halo g76 in snap sn004
g76 already exists
working on halo g1807 in snap sn004
g1807 already exists
working on halo g2017 in snap sn004
g2017 already exists
working on halo g1688 in snap sn004
g1688 already exists
working on halo g187 in snap sn004
working on halo g1306 in snap sn004
g1306 already exists
working on halo g4371 in snap sn004
g4371 already exists
working on halo g2616 in snap sn004
g2616 already exists
working on halo g2291 in snap sn004
g2291 already exists
working 

working on halo g1551 in snap sn004
g1551 already exists
working on halo g4468 in snap sn004
g4468 already exists
working on halo g1317 in snap sn004
g1317 already exists
working on halo g8674 in snap sn004
g8674 already exists
working on halo g473 in snap sn004
g473 already exists
working on halo g707 in snap sn004
g707 already exists
working on halo g4816 in snap sn004
g4816 already exists
working on halo g2391 in snap sn004
g2391 already exists
working on halo g3740 in snap sn004
g3740 already exists
working on halo g433 in snap sn004
g433 already exists
working on halo g7503 in snap sn004
g7503 already exists
working on halo g3854 in snap sn004
g3854 already exists
working on halo g2990 in snap sn004
g2990 already exists
working on halo g2953 in snap sn004
g2953 already exists
working on halo g294 in snap sn004
g294 already exists
working on halo g1747 in snap sn004
g1747 already exists
working on halo g1137 in snap sn004
g1137 already exists
working on halo g2886 in snap sn004
g28

working on halo g5750 in snap sn004
g5750 already exists
working on halo g2814 in snap sn004
g2814 already exists
working on halo g2708 in snap sn004
g2708 already exists
working on halo g814 in snap sn004
g814 already exists
working on halo g5222 in snap sn004
g5222 already exists
working on halo g6334 in snap sn004
g6334 already exists
working on halo g3453 in snap sn004
g3453 already exists
working on halo g6823 in snap sn004
g6823 already exists
working on halo g675 in snap sn004
g675 already exists
working on halo g4192 in snap sn004
g4192 already exists
working on halo g540 in snap sn004
g540 already exists
working on halo g3839 in snap sn004
g3839 already exists
working on halo g1514 in snap sn004
g1514 already exists
working on halo g18560 in snap sn004
g18560 already exists
working on halo g1446 in snap sn004
g1446 already exists
working on halo g1956 in snap sn004
g1956 already exists
working on halo g635 in snap sn004
g635 already exists
working on halo g500 in snap sn004
g5

In [17]:
origin = 'new_main'
destination = 'esc_analysis'
correct_fesc_file(origin=origin, destination=destination)

In [14]:
origin = 'new_main'
destination = 'esc_analysis'
correct_paths(origin=origin, destination=destination)

Working on g18024 in snap sn013
Working on g5025 in snap sn013
Working on g36099 in snap sn013
Working on g3205 in snap sn013
Working on g5444 in snap sn013
Working on g7085 in snap sn013
Working on g5265 in snap sn013
Working on g6715 in snap sn013
Working on g6955 in snap sn013
Working on g14147 in snap sn013
Working on g2324 in snap sn013
Working on g10928 in snap sn013
Working on g34620 in snap sn013
Working on g41356 in snap sn013
Working on g2456 in snap sn013
Working on g13062 in snap sn013
Working on g24614 in snap sn013
Working on g22417 in snap sn013
Working on g29181 in snap sn013
Working on g35962 in snap sn013
Working on g11514 in snap sn013
Working on g5961 in snap sn013
Working on g7003 in snap sn013
Working on g30150 in snap sn013
Working on g3744 in snap sn013
Working on g16482 in snap sn013
Working on g2918 in snap sn013
Working on g2110 in snap sn013
Working on g5899 in snap sn013
Working on g11110 in snap sn013
Working on g11878 in snap sn013
Working on g46321 in sn

Working on g3538 in snap sn013
Working on g3370 in snap sn013
Working on g1016 in snap sn013
Working on g41936 in snap sn013
Working on g23200 in snap sn013
Working on g27076 in snap sn013
Working on g23521 in snap sn013
Working on g20986 in snap sn013
Working on g35535 in snap sn013
Working on g3025 in snap sn013
Working on g5085 in snap sn013
Working on g11133 in snap sn013
Working on g3104 in snap sn013
Working on g22966 in snap sn013
Working on g20117 in snap sn013
Working on g53286 in snap sn013
Working on g8001 in snap sn013
Working on g37055 in snap sn013
Working on g43068 in snap sn013
Working on g22269 in snap sn013
Working on g25601 in snap sn013
Working on g5019 in snap sn013
Working on g2241 in snap sn013
Working on g33311 in snap sn013
Working on g7538 in snap sn013
Working on g3365 in snap sn013
Working on g1545 in snap sn013
Working on g11423 in snap sn013
Working on g1785 in snap sn013
Working on g31048 in snap sn013
Working on g19651 in snap sn013
Working on g23199 in 

Working on g19736 in snap sn013
Working on g3598 in snap sn013
Working on g1869 in snap sn013
Working on g16774 in snap sn013
Working on g12346 in snap sn013
Working on g1166 in snap sn013
Working on g19654 in snap sn013
Working on g17809 in snap sn013
Working on g12646 in snap sn013
Working on g4028 in snap sn013
Working on g10944 in snap sn013
Working on g9307 in snap sn013
Working on g2277 in snap sn013
Working on g7890 in snap sn013
Working on g26384 in snap sn013
Working on g13934 in snap sn013
Working on g21044 in snap sn013
Working on g8544 in snap sn013
Working on g8083 in snap sn013
Working on g3330 in snap sn013
Working on g19543 in snap sn013
Working on g1983 in snap sn013
Working on g3670 in snap sn013
Working on g11951 in snap sn013
Working on g11220 in snap sn013
Working on g69018 in snap sn013
Working on g9737 in snap sn013
Working on g2356 in snap sn013
Working on g13424 in snap sn013
Working on g2038 in snap sn013
Working on g30764 in snap sn013
Working on g8033 in sna

Working on g47783 in snap sn013
Working on g2154 in snap sn013
Working on g9046 in snap sn013
Working on g10922 in snap sn013
Working on g1873 in snap sn013
Working on g7850 in snap sn013
Working on g26305 in snap sn013
Working on g13404 in snap sn013
Working on g20179 in snap sn013
Working on g16752 in snap sn013
Working on g3304 in snap sn013
Working on g40765 in snap sn013
Working on g20364 in snap sn013
Working on g31673 in snap sn013
Working on g29024 in snap sn013
Working on g8245 in snap sn013
Working on g5802 in snap sn013
Working on g2321 in snap sn013
Working on g33715 in snap sn013
Working on g2269 in snap sn013
Working on g4724 in snap sn013
Working on g23496 in snap sn013
Working on g4931 in snap sn013
Working on g9750 in snap sn013
Working on g1874 in snap sn013
Working on g26662 in snap sn013
Working on g15257 in snap sn013
Working on g5998 in snap sn013
Working on g3630 in snap sn013
Working on g21232 in snap sn013
Working on g9922 in snap sn013
Working on g17172 in sna

Working on g8894 in snap sn013
Working on g6889 in snap sn013
Working on g26810 in snap sn013
Working on g2654 in snap sn013
Working on g14835 in snap sn013
Working on g7563 in snap sn013
Working on g18534 in snap sn013
Working on g1495 in snap sn013
Working on g26313 in snap sn013
Working on g7297 in snap sn013
Working on g6174 in snap sn013
Working on g27086 in snap sn013
Working on g6477 in snap sn013
Working on g4131 in snap sn013
Working on g13816 in snap sn013
Working on g9196 in snap sn013
Working on g8707 in snap sn013
Working on g3175 in snap sn013
Working on g39193 in snap sn013
Working on g3509 in snap sn013
Working on g25068 in snap sn013
Working on g28866 in snap sn013
Working on g15928 in snap sn013
Working on g1480 in snap sn013
Working on g4366 in snap sn013
Working on g7594 in snap sn013
Working on g4204 in snap sn013
Working on g56124 in snap sn013
Working on g4449 in snap sn013
Working on g20895 in snap sn013
Working on g24552 in snap sn013
Working on g1455 in snap s

...

Working on g1276 in snap sn004
Working on g1786 in snap sn004
Working on g1121 in snap sn004
Working on g248 in snap sn004
Working on g113 in snap sn004
Working on g5719 in snap sn004
Working on g1842 in snap sn004
Working on g807 in snap sn004
Working on g7599 in snap sn004
Working on g2833 in snap sn004
Working on g533 in snap sn004
Working on g1867 in snap sn004
Working on g2077 in snap sn004
Working on g11691 in snap sn004
Working on g580 in snap sn004
Working on g394 in snap sn004
Working on g628 in snap sn004
Working on g4333 in snap sn004
Working on g1231 in snap sn004
Working on g17161 in snap sn004
Working on g307 in snap sn004
Working on g4282 in snap sn004
Working on g953 in snap sn004
Working on g18705 in snap sn004
Working on g489 in snap sn004
Working on g354 in snap sn004
Working on g20548 in snap sn004
Working on g1761 in snap sn004
Working on g168 in snap sn004
Working on g1784 in snap sn004
Working on g4548 in snap sn004
Working on g913 in snap sn004
Working on g1687 

Working on g7402 in snap sn004
Working on g962 in snap sn004
Working on g5994 in snap sn004
Working on g10396 in snap sn004
Working on g2604 in snap sn004
Working on g1750 in snap sn004
Working on g3451 in snap sn004
Working on g10905 in snap sn004
Working on g75 in snap sn004
Working on g177 in snap sn004
Working on g4047 in snap sn004
Working on g21354 in snap sn004
Working on g1068 in snap sn004
Working on g736 in snap sn004
Working on g601 in snap sn004
Working on g4291 in snap sn004
Working on g71 in snap sn004
Working on g2407 in snap sn004
Working on g137 in snap sn004
Working on g28306 in snap sn004
Working on g783 in snap sn004
Working on g12528 in snap sn004
Working on g597 in snap sn004
Working on g8739 in snap sn004
Working on g3226 in snap sn004
Working on g462 in snap sn004
Working on g1349 in snap sn004
Working on g2289 in snap sn004
Working on g557 in snap sn004
Working on g422 in snap sn004
Working on g236 in snap sn004
Working on g101 in snap sn004
Working on g882 in 

Working on g2285 in snap sn004
Working on g1648 in snap sn004
Working on g3590 in snap sn004
Working on g1900 in snap sn004
Working on g745 in snap sn004
Working on g1583 in snap sn004
Working on g1213 in snap sn004
Working on g610 in snap sn004
Working on g559 in snap sn004
Working on g2385 in snap sn004
Working on g424 in snap sn004
Working on g471 in snap sn004
Working on g2182 in snap sn004
Working on g2005 in snap sn004
Working on g7382 in snap sn004
Working on g1496 in snap sn004
Working on g5573 in snap sn004
Working on g285 in snap sn004
Working on g519 in snap sn004
Working on g8689 in snap sn004
Working on g1041 in snap sn004
Working on g1818 in snap sn004
Working on g3577 in snap sn004
Working on g7842 in snap sn004
Working on g3164 in snap sn004
Working on g1533 in snap sn004
Working on g5251 in snap sn004
Working on g3818 in snap sn004
Working on g245 in snap sn004
Working on g110 in snap sn004
Working on g891 in snap sn004
Working on g2020 in snap sn004
Working on g9958 i

Working on g2990 in snap sn004
Working on g2953 in snap sn004
Working on g294 in snap sn004
Working on g1747 in snap sn004
Working on g1137 in snap sn004
Working on g2886 in snap sn004
Working on g207 in snap sn004
Working on g5341 in snap sn004
Working on g853 in snap sn004
Working on g3573 in snap sn004
Working on g1083 in snap sn004
Working on g4132 in snap sn004
Working on g389 in snap sn004
Working on g4732 in snap sn004
Working on g8638 in snap sn004
Working on g254 in snap sn004
Working on g1439 in snap sn004
Working on g2903 in snap sn004
Working on g3192 in snap sn004
Working on g948 in snap sn004
Working on g3050 in snap sn004
Working on g813 in snap sn004
Working on g7187 in snap sn004
Working on g627 in snap sn004
Working on g3052 in snap sn004
Working on g8252 in snap sn004
Working on g1567 in snap sn004
Working on g1420 in snap sn004
Working on g674 in snap sn004
Working on g3512 in snap sn004
Working on g4727 in snap sn004
Working on g1159 in snap sn004
Working on g4865 

Working on g815 in snap sn004


In [52]:
origin = '/raven/ptmp/ivkos/new_5e-1'
destination = '/ptmp/mpa/mglatzle/TNG_f_esc/new_5e-1'
correct_fesc_file(origin, destination, data_dir='/ptmp/mpa/mglatzle/TNG_f_esc', from_freya=True)

In [17]:
origin = '/raven/ptmp/ivkos/new_dust'
destination = '/ptmp/mpa/mglatzle/TNG_f_esc/new_dust'
correct_paths(origin=origin, with_dust=True, destination=destination)

Working on g61 in snap sn013
Working on g13697 in snap sn013
Working on g48185 in snap sn013
Working on g13632 in snap sn013
Working on g936 in snap sn013
Working on g202 in snap sn013
Working on g21316 in snap sn013
Working on g4096 in snap sn013
Working on g13393 in snap sn013
Working on g41 in snap sn013
Working on g11902 in snap sn013
Working on g15689 in snap sn013
Working on g823 in snap sn013
Working on g502 in snap sn013
Working on g3538 in snap sn013
Working on g184 in snap sn013
Working on g3125 in snap sn013
Working on g3081 in snap sn013
Working on g838 in snap sn013
Working on g2716 in snap sn013
Working on g14510 in snap sn013
Working on g250 in snap sn013
Working on g159 in snap sn013
Working on g14152 in snap sn013
Working on g15825 in snap sn013
Working on g5535 in snap sn013
Working on g404 in snap sn013
Working on g3553 in snap sn013
Working on g130 in snap sn013
Working on g34927 in snap sn013
Working on g64873 in snap sn013
Working on g5887 in snap sn013
Working on

Working on g24903 in snap sn013
Working on g1801 in snap sn013
Working on g269 in snap sn013
Working on g30778 in snap sn013
Working on g4498 in snap sn013
Working on g280 in snap sn013
Working on g13121 in snap sn013
Working on g1544 in snap sn013
Working on g2322 in snap sn013
Working on g35658 in snap sn013
Working on g27145 in snap sn013
Working on g59 in snap sn013
Working on g160 in snap sn013
Working on g1619 in snap sn013
Working on g20126 in snap sn013
Working on g960 in snap sn013
Working on g47 in snap sn013
Working on g869 in snap sn013
Working on g89 in snap sn013
Working on g10485 in snap sn013
Working on g6404 in snap sn013
Working on g18435 in snap sn013
Working on g28453 in snap sn013
Working on g982 in snap sn013
Working on g32874 in snap sn013
Working on g3959 in snap sn013
Working on g482 in snap sn013
Working on g10984 in snap sn013
Working on g121 in snap sn013
Working on g10398 in snap sn013
Working on g1825 in snap sn008
Working on g2519 in snap sn008
Working on

Working on g328 in snap sn008
Working on g240 in snap sn008
Working on g149 in snap sn008
Working on g2499 in snap sn008
Working on g59 in snap sn008
Working on g9516 in snap sn008
Working on g120 in snap sn008
Working on g17317 in snap sn008
Working on g1118 in snap sn008
Working on g449 in snap sn008
Working on g314 in snap sn008
Working on g960 in snap sn008
Working on g3559 in snap sn008
Working on g13228 in snap sn008
Working on g5136 in snap sn008
Working on g35 in snap sn008
Working on g3674 in snap sn008
Working on g982 in snap sn008
Working on g1896 in snap sn008
Working on g756 in snap sn008
Working on g73 in snap sn008
Working on g435 in snap sn008
Working on g5165 in snap sn008
Working on g23 in snap sn008
Working on g815 in snap sn008
Working on g1146 in snap sn004
Working on g1162 in snap sn004
Working on g65 in snap sn004
Working on g5564 in snap sn004
Working on g322 in snap sn004
Working on g11 in snap sn004
Working on g742 in snap sn004
Working on g53 in snap sn004
Wo

In [30]:
origin = 'new_main'
destination = 'esc_analysis'
correct_fesc_file(origin=origin, destination=destination)

In [11]:
testpath = '/ptmp/mpa/mglatzle/TNG_f_esc/esc_analysis/run/L35n2160TNG/sn008/g1975/SYS_INIT.in'
print(origin not in os.path.realpath(testpath))

True


In [20]:
def remove_halo(destination, snaps, halos, data_dir='/ptmp/mpa/mglatzle/TNG_f_esc'):
    esc_file = os.path.join(data_dir, destination, '00_f_esc.txt')
    file = open(esc_file, "r")
    file_new = open(esc_file+'_new', 'w')
    for snap in snaps:
        for halo in halos:
            for line in file:
                if f'{snap}/g{halo}' not in line:
                    file_new.write(line)

    file.close()
    file_new.close()

In [10]:
from crashpy.dataclasses.simulation import LoadedSimulation as Sim

In [5]:
broken_paths = []

In [12]:
with open('/ptmp/mpa/mglatzle/TNG_f_esc/esc_analysis/00_f_esc.txt', 'r') as f:
    sims = [x.strip() for x in f.readlines()]
for i, simpath in enumerate(sims):
    try:
        sim = Sim(simpath)
        del sim
    except:
        broken_paths.append(simpath)
        print(simpath)
        continue

/freya/ptmp/mpa/mglatzle/TNG_f_esc/esc_analysis/run/L35n2160TNG/sn008/g8154
/freya/ptmp/mpa/mglatzle/TNG_f_esc/esc_analysis/run/L35n2160TNG/sn008/g2069
/freya/ptmp/mpa/mglatzle/TNG_f_esc/esc_analysis/run/L35n2160TNG/sn008/g3619


In [22]:
remove_halo('esc_analysis', ['sn008'], [8154, 2069, 3619], data_dir='/ptmp/mpa/mglatzle/TNG_f_esc')

In [13]:
broken_paths

['/freya/ptmp/mpa/mglatzle/TNG_f_esc/esc_analysis/run/L35n2160TNG/sn013/g18024',
 '/freya/ptmp/mpa/mglatzle/TNG_f_esc/esc_analysis/run/L35n2160TNG/sn013/g5025',
 '/freya/ptmp/mpa/mglatzle/TNG_f_esc/esc_analysis/run/L35n2160TNG/sn013/g36099',
 '/freya/ptmp/mpa/mglatzle/TNG_f_esc/esc_analysis/run/L35n2160TNG/sn013/g3205',
 '/freya/ptmp/mpa/mglatzle/TNG_f_esc/esc_analysis/run/L35n2160TNG/sn013/g5444',
 '/freya/ptmp/mpa/mglatzle/TNG_f_esc/esc_analysis/run/L35n2160TNG/sn013/g7085',
 '/freya/ptmp/mpa/mglatzle/TNG_f_esc/esc_analysis/run/L35n2160TNG/sn013/g5265',
 '/freya/ptmp/mpa/mglatzle/TNG_f_esc/esc_analysis/run/L35n2160TNG/sn013/g6715',
 '/freya/ptmp/mpa/mglatzle/TNG_f_esc/esc_analysis/run/L35n2160TNG/sn013/g6955',
 '/freya/ptmp/mpa/mglatzle/TNG_f_esc/esc_analysis/run/L35n2160TNG/sn013/g14147',
 '/freya/ptmp/mpa/mglatzle/TNG_f_esc/esc_analysis/run/L35n2160TNG/sn013/g2324',
 '/freya/ptmp/mpa/mglatzle/TNG_f_esc/esc_analysis/run/L35n2160TNG/sn013/g10928',
 '/freya/ptmp/mpa/mglatzle/TNG_f_esc

In [60]:
broken = [2133, 2459, 1158, 1057, 1667, 121, 1519]

[2133, 2459, 1158, 1057, 1667, 121]

In [95]:
broken008 = [2069, 3819, 1132]

In [62]:
remove_halo('esc_analysis', 'sn004', '1519')

In [56]:
correct_fesc_file(origin, destination, data_dir='/ptmp/mpa/mglatzle/TNG_f_esc', from_freya=True)

In [63]:
from crashpy.dataclasses.simulation import LoadedSimulation as Sim

In [86]:
import pickle

In [88]:
with open('now_working', 'rb') as f:
    work_list = pickle.load(f)

In [89]:
work_list

['/freya/ptmp/mpa/mglatzle/TNG_f_esc/esc_analysis/run/L35n2160TNG/sn008/g3619',
 '/freya/ptmp/mpa/mglatzle/TNG_f_esc/esc_analysis/run/L35n2160TNG/sn008/g1132']

In [96]:
path = '/freya/ptmp/mpa/mglatzle/TNG_f_esc/esc_analysis/run/L35n2160TNG/sn008/g1293'

In [97]:
csim = Sim(path)